# Week 3

## Task 1 - Optical flow

### Task 1.1: Optical flow estimation with off-the-shelf method

Quantitative results:
- MSEN: Mean Square Error in Non-occluded areas
- PEPN: Percentage of Erroneous Pixels in Non-occluded areas
- Runtime 

Dataset: Sequences 45 (image_0) from [KITTI](https://www.cvlibs.net/datasets/kitti/eval_stereo_flow.php?benchmark=stereo) (Click "Download stereo/optical flow data set (2 GB)").

In [ ]:
def compute_msen(flow_gt, flow_est, valid_mask):
    diff = flow_gt - flow_est
    epe = np.sqrt(np.sum(diff**2, axis=2))
    return np.mean(epe[valid_mask])

def compute_pepn(flow_gt, flow_est, valid_mask, threshold=3.0):
    diff = flow_gt - flow_est
    epe = np.sqrt(np.sum(diff**2, axis=2))
    err_pixels = np.sum((epe > threshold) & valid_mask)
    total_valid = np.sum(valid_mask)
    return 100.0 * err_pixels / total_valid

In [ ]:
from pathlib import Path

import cv2
import numpy as np
import matplotlib.pyplot as plt

# Paths to KITTI data
DATASET_PATH = Path("data_stereo_flow/training/")
IMG_PATH = DATASET_PATH / "image_0"
FLOW_PATH = DATASET_PATH / "flow_occ"
assert DATASET_PATH.exists()
assert IMG_PATH.exists()
assert FLOW_PATH.exists()

# Load images
img1 = cv2.imread(IMG_PATH / "000045_10.png", cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread(IMG_PATH / "000045_11.png", cv2.IMREAD_GRAYSCALE)

# Load ground truth flow
flow_gt = cv2.imread(FLOW_PATH / "000045_10.png", cv2.IMREAD_UNCHANGED)

# KITTI encodes flow as uint16, need to decode it
u = (flow_gt[:, :, 2].astype(np.float32) - 2**15) / 64.0
v = (flow_gt[:, :, 1].astype(np.float32) - 2**15) / 64.0
valid_mask = flow_gt[:, :, 0] > 0
flow_gt = np.dstack((u, v))

In [ ]:
# Visualize the images and flow side by side
plt.figure(figsize=(15, 5))

plt.subplot(2, 2, 1)
plt.imshow(img1, cmap='gray')
plt.title("Image 1 (t)")

plt.subplot(2, 2, 2)
plt.imshow(img2, cmap='gray')
plt.title("Image 2 (t+1)")

# Downsample for visualization (to avoid too many arrows)
step = 10
h, w = u.shape
y, x = np.mgrid[0:h:step, 0:w:step]
u_down = u[::step, ::step]
v_down = v[::step, ::step]
valid_down = valid_mask[::step, ::step]

plt.subplot(2, 2, 3)
plt.imshow(img1, cmap='gray')
plt.quiver(x, y, u_down * valid_down, v_down * valid_down, color='r', angles='xy', scale_units='xy', scale=0.5)
plt.title("Optical Flow (GT)")

# Flow estimation, assuming flow_gt is available from pyflow or another flow estimation method
# flow_gt is a 3D array with shape (height, width, 2) representing the flow components (u, v)
# For example, you could use: u, v, _ = pyflow.coarse2fine_flow(...)

# Prepare flow visualization in HSV
flow_hsv = np.zeros((img1.shape[0], img1.shape[1], 3), dtype=np.uint8)
flow_hsv[:, :, 0] = 255  # Set Hue to 255 (will adjust based on angle)
flow_hsv[:, :, 1] = 255  # Set Saturation to 255 (full saturation)
# Assuming flow_gt contains the flow components u (horizontal) and v (vertical)
mag, ang = cv2.cartToPolar(flow_gt[..., 0], flow_gt[..., 1])
# Map angle (ang) to hue in the range [0, 180]
flow_hsv[..., 0] = (ang * 180 / np.pi / 2).astype(np.uint8)
# Normalize magnitude to [0, 255] and use it for Value channel
flow_hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
# Convert HSV to RGB
flow_rgb = cv2.cvtColor(flow_hsv, cv2.COLOR_HSV2BGR)

plt.subplot(2, 2, 4)
plt.imshow(flow_rgb, vmin=flow_rgb.min(), vmax=flow_rgb.max())
plt.title("Optical Flow (GT)")

plt.tight_layout()
plt.show()

Compute and asses the optical flow provided in [PyFlow](https://github.com/pathak22/pyflow).

In [ ]:
from pyflow import pyflow

In [ ]:
# Estimate flow using PyFlow
alpha = 0.012
ratio = 0.75
minWidth = 20
nOuterFPIterations = 7
nInnerFPIterations = 1
nSORIterations = 30
colType = 1  # 0 or default:RGB, 1:GRAY (but pass gray image with shape (h,w,1))

u, v, im2W = pyflow.coarse2fine_flow(
    img1[..., np.newaxis].astype(float) / 255., img2[..., np.newaxis].astype(float) / 255.,
    alpha, ratio, minWidth, nOuterFPIterations, nInnerFPIterations,
    nSORIterations, colType
)
flow_est = np.dstack((u, v))

In [ ]:
# Visualize the images and flow side by side
plt.figure(figsize=(15, 5))

plt.subplot(2, 2, 1)
plt.imshow(img1, cmap='gray')
plt.title("Image 1 (t)")

plt.subplot(2, 2, 2)
plt.imshow(img2, cmap='gray')
plt.title("Image 2 (t+1)")

# Downsample for visualization (to avoid too many arrows)
step = 10
h, w = u.shape
y, x = np.mgrid[0:h:step, 0:w:step]
u_down = u[::step, ::step]
v_down = v[::step, ::step]

plt.subplot(2, 2, 3)
plt.imshow(img1, cmap='gray')
plt.quiver(x, y, u_down, v_down, color='r', angles='xy', scale_units='xy', scale=0.5)
plt.title("Optical Flow (GT)")

# Flow estimation, assuming flow_est is available from pyflow or another flow estimation method
# flow_est is a 3D array with shape (height, width, 2) representing the flow components (u, v)
# For example, you could use: u, v, _ = pyflow.coarse2fine_flow(...)

# Prepare flow visualization in HSV
flow_hsv = np.zeros((img1.shape[0], img1.shape[1], 3), dtype=np.uint8)
flow_hsv[:, :, 0] = 255  # Set Hue to 255 (will adjust based on angle)
flow_hsv[:, :, 1] = 255  # Set Saturation to 255 (full saturation)
# Assuming flow_est contains the flow components u (horizontal) and v (vertical)
mag, ang = cv2.cartToPolar(flow_est[..., 0], flow_est[..., 1])
# Map angle (ang) to hue in the range [0, 180]
flow_hsv[..., 0] = (ang * 180 / np.pi / 2).astype(np.uint8)
# Normalize magnitude to [0, 255] and use it for Value channel
flow_hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
# Convert HSV to RGB
flow_rgb = cv2.cvtColor(flow_hsv, cv2.COLOR_HSV2BGR)

plt.subplot(2, 2, 4)
plt.imshow(flow_rgb, vmin=flow_rgb.min(), vmax=flow_rgb.max())
plt.title("Optical Flow (GT)")

plt.tight_layout()
plt.show()

In [ ]:
# Calculate metrics
msen = compute_msen(flow_gt, flow_est, valid_mask)
pepn = compute_pepn(flow_gt, flow_est, valid_mask)

print(f"MSEN: {msen:.2f}")
print(f"PEPN: {pepn:.2f}%")

Search and asses another optical flow implementation. For each referred technique, specify:
- Citation to the related paper.
- Link to an external implementation or specify if you wrote your own one.